<a href="https://colab.research.google.com/github/MuPeres/MotorFlawPrediction/blob/main/Final_Work_MuriloFurlan_CD_Trabalho2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Descrição do Problema


##Contextualização

Detecção de Desbalanceamento em Motores de Indução

Os motores de indução desempenham um papel crucial em inúmeras aplicações industriais, sendo a força motriz por trás de maquinários que impulsionam a produção e operação eficientes. No entanto, sua integridade operacional frequentemente enfrenta desafios, e entre os problemas recorrentes destaca-se o desbalanceamento do rotor.

O desbalanceamento em motores de indução é uma condição crítica que ocorre quando a distribuição de massa no rotor não é uniforme. Essa disparidade resulta em um centro de gravidade não alinhado com o eixo de rotação, provocando vibrações indesejadas durante o funcionamento do motor. Existem várias razões pelas quais o desbalanceamento pode ocorrer, e sua detecção precoce é vital para evitar consequências adversas.

##Causas do Desbalanceamento

Desgaste Desigual:
O desbalanceamento pode surgir devido ao desgaste desigual dos componentes do motor, especialmente nos rolamentos. O desgaste assimétrico leva a uma distribuição de massa irregular, desencadeando o desbalanceamento.

Acúmulo de Material: A acumulação de material estranho no rotor, como poeira ou resíduos, pode resultar em desbalanceamento. O acúmulo gradual altera a distribuição de massa, impactando diretamente o equilíbrio do motor.

Variações na Produção:
Variações no processo de fabricação podem introduzir diferenças mínimas na distribuição de massa entre rotores. Embora essas variações sejam muitas vezes insignificantes, podem se manifestar como desbalanceamento durante a operação.

##Consequências do Desbalanceamento

O desbalanceamento representa uma condição crítica que merece atenção especial. Quando não tratado, pode levar a uma série de problemas graves:

Danos aos Rolamentos:
Vibrações intensas provenientes do desbalanceamento exercem pressão adicional sobre os rolamentos, resultando em desgaste acelerado e, eventualmente, falhas.

Desgaste Prematuro de Componentes:
O desbalanceamento aumenta as cargas mecânicas nos componentes do motor, levando a um desgaste prematuro. Isso não apenas compromete a eficiência do motor, mas também eleva os custos de manutenção.

Perda de Eficiência:
O aumento das vibrações não apenas compromete a integridade dos componentes, mas também reduz a eficiência global do motor. Isso se reflete em um aumento no consumo de energia e na diminuição do desempenho.

Riscos de Falhas Catastróficas:
O desbalanceamento, se não tratado, pode evoluir para falhas catastróficas do motor, resultando em paradas inesperadas da produção e custos substanciais de reparo.

A detecção atempada do desbalanceamento emerge como uma peça fundamental na manutenção preditiva de motores de indução. Identificar e corrigir desvios na distribuição de massa antes que evoluam para problemas mais sérios é essencial para prolongar a vida útil do motor e otimizar os custos operacionais. Entre os diversos cenários de desbalanceamento, nosso foco recai sobre o desbalanceamento de uma massa de 25g, uma condição crítica que exige atenção especial. Devido a massa mais notável, sua detecção se torna mais fácil que de massas menores, como a de 10g. Entretanto, apesar da massa de desbalanceamento ser elevada, o motor mantém sua rotação de funcionamento normalmente, o que não acontece para massas superiores à 30g.

##Importância da Detecção de Desbalanceamento

O desbalanceamento representa um desafio significativo, não apenas devido à sua magnitude, mas também pelos impactos severos que pode causar. Essa condição, se não identificada e corrigida rapidamente, pode resultar em danos consideráveis aos rolamentos, eixos e outros componentes do motor. Além disso, o aumento das vibrações pode acelerar o desgaste mecânico, reduzindo drasticamente a eficiência do motor.

Neste contexto, a aplicação de técnicas avançadas de aprendizado de máquina ganha destaque como uma abordagem proativa para a detecção de desbalanceamento. A análise de dados provenientes de sensores, como tacômetros, acelerômetros e microfones, oferece uma oportunidade única de identificar padrões sutis associados ao desbalanceamento. A implementação de modelos preditivos não apenas permite uma resposta rápida a condições de desbalanceamento, mas também serve como base para estratégias de manutenção preditiva, reduzindo os custos operacionais e maximizando a confiabilidade dos motores de indução.

## Objetivo

Este trabalho visa desenvolver um modelo de detecção de desbalanceamento em motores de indução, com foco na condição específica de desbalanceamento de 25g. Além disso, busca-se realizar uma comparação entre diferentes técnicas de processamento para otimizar a identificação desse problema, visando aprimorar a confiabilidade e eficácia do processo de detecção.

#Pré-Processamento
###Inicialmente, vamos aqui chamar as bibliotecas utilizadas, além de organizar as funções que serão utilizadas e iniciar o programa. Em seguida, serão estudados dois baselines:
1. O primeiro com os dados brutos no domínio do tempo;
2. O segundo com os dados no domínio da frequência.

O primeiro Baseline irá aplicar o treino do modelo a partir do *knn*, e através da função



## 1.0 Bibliotecas

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from scipy.fft import fft

## 1.1 Função para carregar os dados
Carrega dados de arquivos CSV em uma lista de DataFrames.

In [ ]:
def load_data(folder_path):
    data = []
    for filename in os.listdir(folder_path):
        if filename.endswith('.csv'):
            file_path = os.path.join(folder_path, filename)
            df = pd.read_csv(file_path, header=None)
            data.append(df)
    return data

## 1.2 Função para pré-processar dados
Realiza o pré-processamento dos dados, selecionando colunas específicas e adicionando rótulos.

In [ ]:
def preprocess_data(data, label):
    processed_data = []
    for i in range(len(data)):
        df = data[i].iloc[:, 7:8].T
        df['label'] = label
        processed_data.append(df)
    return processed_data

##1.3 Função de plot para várias matrizes confusão:
Gera subgráficos de matrizes de confusão para diferentes valores de k.

In [ ]:
def plot_confusion_matrices(k_values, conf_matrices):
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=(15, 10))
    fig.suptitle('Matriz de Confusão para Diferentes Valores de K')
    for i, (k, conf_matrix) in enumerate(zip(k_values, conf_matrices), 1):
        ax = plt.subplot(2, 3, i)
        cax = ax.matshow(conf_matrix, cmap=plt.cm.Blues)
        plt.title(f'K = {k}')
        plt.colorbar(cax)
        plt.xlabel('Previsto')
        plt.ylabel('Verdadeiro')
    plt.show()

## 1.4 Função de extração de features
 Extrai características, amplitude média e frequência dominante, de uma linha.

In [ ]:
def extract_features(row):
    # Amplitude Média
    amplitude_media = row.iloc[1:].mean()  # A média das colunas, excluindo a primeira (label)

    # Frequência Dominante
    signal = row.iloc[1:-1].values  # Excluindo a primeira (label) e a última coluna
    fft_result = fft(signal)
    fft_magnitude = np.abs(fft_result)
    dominant_frequency_index = np.argmax(fft_magnitude[1:]) + 1  # Ignorar a componente DC
    dominant_frequency = dominant_frequency_index / len(signal)
    return pd.Series([amplitude_media, dominant_frequency], index=['Amplitude_Media', 'Frequencia_Dominante'])

#1.5 Avaliar o KNN
Treina o modelo k-NN para diferentes valores de k e avalia o desempenho.

In [ ]:
def train_and_evaluate_knn(X_train, y_train, X_test, y_test, k_values):
    accuracies = []
    conf_matrices = []

    for k in k_values:
        knn = KNeighborsClassifier(n_neighbors=k)
        knn.fit(X_train, y_train)
        y_pred_k = knn.predict(X_test)

        accuracy_k = accuracy_score(y_test, y_pred_k)
        accuracies.append(accuracy_k)

        conf_matrix_k = confusion_matrix(y_test, y_pred_k)
        conf_matrices.append(conf_matrix_k)
    return accuracies, conf_matrices

#1.6 Visualizar acurácias
Gera um gráfico da curva de acurácia em função dos valores de k.

In [ ]:
def plot_accuracy_curve(k_values, accuracies):
    plt.figure()
    plt.plot(k_values, accuracies, marker='o')
    plt.title('Acurácia em função de K')
    plt.xlabel('Valor de K')
    plt.ylabel('Acurácia')
    plt.show()

#1.7 FFT
Converte os dados para o domínio da frequência usando a Transformada Rápida de Fourier (FFT).

In [ ]:
def convert_to_frequency_domain(row):
    signal = row.iloc[:-1].values  # Excluindo a última coluna (label)
    fft_result = fft(signal)
    return pd.Series(np.abs(fft_result[1:]), name='Freq_Domain')

# Inicialização
Para a iniciação do programa, depois de acessados os arquivos, será selecionada uma coluna das 8 presentes em cada arquivo csv. Nesse caso, foi selecionada a coluna de dados do som (coluna 7). Em seguida, cada base de dados, agora composta por uma coluna, será transposta, e adicionada de uma coluna para "label", identificando os motores com falha e não falha. A partir desse ponto, cada baseline será trabalhado separadamente com a sua diferente proposta.

In [ ]:
# Pastas contendo os dados dos motores
folder_path_normal = r'C:\Users\lilop\OneDrive - UNIOESTE\Documentos\Python Scripts\CD2023\normal\normal'
folder_path_faulty = r'C:\Users\lilop\OneDrive - UNIOESTE\Documentos\Python Scripts\CD2023\imbalance\25g'

# Carregar os dados
data_normal = load_data(folder_path_normal)
data_faulty = load_data(folder_path_faulty)

# Pré-processamento dos dados
data_normal_processed = preprocess_data(data_normal, 0)
data_faulty_processed = preprocess_data(data_faulty, 1)

# Concatenar os DataFrames
all_data = pd.concat(data_normal_processed + data_faulty_processed, ignore_index=True)
# Reorganizar as colunas para colocar 'label' no início
all_data = all_data[['label'] + list(all_data.columns[:-1])]

# Normalização
scaler = MinMaxScaler()
all_data.iloc[:, 1:] = scaler.fit_transform(all_data.iloc[:, 1:])

# Cenário 1 - Dados brutos

In [ ]:
# Treino
X = all_data.drop(['label'], axis=1)
y = all_data['label']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Avaliando o K
k_values = [1, 3, 5, 7, 9]
accuracies, conf_matrices = train_and_evaluate_knn(X_train, y_train, X_test, y_test, k_values)
plot_accuracy_curve(k_values, accuracies)
best_k = k_values[accuracies.index(max(accuracies))]
accuracy_best = max(accuracies)
print("Cenário1- Melhor valor de k:", best_k)
print("Cenário1- Acurácia do modelo com o melhor valor de k:", accuracy_best)

# Plotar matrizes de confusão
plot_confusion_matrices(k_values, conf_matrices)

#Cenário 2- Domínio da Frequência

In [ ]:
freq_domain_data = all_data.apply(convert_to_frequency_domain, axis=1) # Aplicar a transformação para o domínio da frequência
# Treino
X_freq = freq_domain_data  # Utilizando os dados transformados para o domínio da frequência
y_freq = all_data['label']
X_train_freq, X_test_freq, y_train_freq, y_test_freq = train_test_split(X_freq, y_freq, test_size=0.3, random_state=42)

# Avaliando o K
accuracies_freq, conf_matrices_freq = train_and_evaluate_knn(X_train_freq, y_train_freq, X_test_freq, y_test_freq, k_values)
plot_accuracy_curve(k_values, accuracies_freq)
best_k_freq = k_values[accuracies_freq.index(max(accuracies_freq))]
accuracy_best_freq = max(accuracies_freq)
print("Cenário2- Melhor valor de k:", best_k_freq)
print("Cenário2- Acurácia do modelo com o melhor valor de k:", accuracy_best_freq)

# Plotar matrizes de confusão
plot_confusion_matrices(k_values, conf_matrices_freq)

#Cenário 3- Extração de Features

In [ ]:
features = all_data.apply(extract_features, axis=1)
all_data = pd.concat([all_data, features], axis=1)  # Adicionar as novas características ao DataFrame
# Treino
X_feat = all_data[['Amplitude_Media', 'Frequencia_Dominante']]
y_feat = all_data['label']
X_train_feat, X_test_feat, y_train_feat, y_test_feat = train_test_split(X_feat, y_feat, test_size=0.3, random_state=42)

# Avaliando o K
accuracies_feat, conf_matrices_feat = train_and_evaluate_knn(X_train_feat, y_train_feat, X_test_feat, y_test_feat, k_values)
plot_accuracy_curve(k_values, accuracies_feat)
best_k_feat = k_values[accuracies_feat.index(max(accuracies_feat))]
accuracy_best_feat = max(accuracies_feat)
print("Cenário3- Melhor valor de k:", best_k_feat)
print("Cenário3- Acurácia do modelo com o melhor valor de k:", accuracy_best_feat)

#Teste estatístico

In [ ]:
# Avaliações de cada cenário
accuracies_cen1 = accuracies
accuracies_cen2 = accuracies_freq
accuracies_cen3 = accuracies_feat

# Realiza o teste t-pareado entre cada cenário
t_statistic_12, p_value_12 = ttest_rel(accuracies_cen1, accuracies_cen2) # 1 e 2
t_statistic_13, p_value_13 = ttest_rel(accuracies_cen1, accuracies_cen3) # 1 e 3
t_statistic_23, p_value_23 = ttest_rel(accuracies_cen2, accuracies_cen3) # 2 e 3

# Imprime os resultados
print(f"Teste t-pareado entre Cenário 1 e Cenário 2: p-value = {p_value_12}")
print(f"Teste t-pareado entre Cenário 1 e Cenário 3: p-value = {p_value_13}")
print(f"Teste t-pareado entre Cenário 2 e Cenário 3: p-value = {p_value_23}")

Os cenários 1 e 3 mostram diferenças significativas em relação ao Cenário 2, mas entre eles próprios, apenas o Cenário 1 e Cenário 3 mostram uma diferença estatisticamente significativa. Isso sugere que o Cenário 3 (com extração de características) pode oferecer um desempenho estatisticamente significativo em comparação com os outros dois.
Entretanto, o Cenário 2 pode ser mais vantajoso para aplicações práticas, pois ofereceu a acurácia mais alta e apresentou diferenças significativas em comparação com o Cenário 1.